## Word2Vec Modeling

This notebook follows on the [non-neural network notebook](https://github.com/kcalizadeh/phil_nlp/blob/master/Notebooks/2_non-neural_models.ipynb). Here, however, we leave aside classificatory modeling and instead aim to understand how the different schools use their words. We achieve this by using word2vec. Initial results trained on the school in isolation were not very promising. But after importing the GloVe pre-trained vectors, we were able to get a compelling picture of how each school uses key terms. 

### Imports and Mounting Drive

In [1]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

# install relevent libraries not included with colab
!pip install lime

drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

     |████████████████████████████████| 276kB 13.7MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=290015c2bd2d8aa5c9a9bdfde358b2ddc2ba83efd753ad878000ab8dd381560d
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
Mounted at /gdrive


In [2]:
from functions import *
%load_ext autoreload
%autoreload 2

np.random_seed=17

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load the Data

In [3]:
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/Phil_NLP/phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
34917,Plato - Complete Works,Plato,plato,"However, there remains the view that they can ...","However, there remains the view that they can ...",84,"however, there remains the view that they can ...","['however', 'there', 'remains', 'the', 'view',...","however , there remain the view that -PRON- c..."
245359,Off The Beaten Track,Heidegger,phenomenology,lated iiber gesetzt us?,lated iiber gesetzt us?,23,lated iiber gesetzt us?,"['lated', 'iiber', 'gesetzt', 'us']",lated iiber gesetzt -PRON- ?
278996,The Phenomenology Of Spirit,Hegel,german_idealism,I perceive in them the free. .),I perceive in them the free. .),31,i perceive in them the free. .),"['perceive', 'in', 'them', 'the', 'free']",-PRON- perceive in -PRON- the free . . )
213589,Anti-Oedipus,Deleuze,continental,"It is the thing, unnamable, the generalized de...","It is the thing, unnamable, the generalized de...",207,"it is the thing, unnamable, the generalized de...","['it', 'is', 'the', 'thing', 'unnamable', 'the...","-PRON- be the thing , unnamable , the general..."
196800,The Order Of Things,Foucault,continental,But this equality does not mean that one excha...,But this equality does not mean that one excha...,277,but this equality does not mean that one excha...,"['but', 'this', 'equality', 'does', 'not', 'me...",but this equality do not mean that one exchan...


In [15]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

In [20]:
# check how it worked
print(df.iloc[290646]['sentence_str'])
df['gensim_tokenized'][290646]

A spider conducts operations that resemble those of a weaver, and a bee puts to shame many an architect in the construction of her cells.


['spider',
 'conducts',
 'operations',
 'that',
 'resemble',
 'those',
 'of',
 'weaver',
 'and',
 'bee',
 'puts',
 'to',
 'shame',
 'many',
 'an',
 'architect',
 'in',
 'the',
 'construction',
 'of',
 'her',
 'cells']

Hmm, an interesting observation. 

For our w2v models, first we'll focus on a single school, since a single school is more likely to have consistency in their use of a word.

Unfortunately, we didn't have much luck with just training on the texts alone. The code for it is left here for posterity, but it was when we worked with GloVe as the base that we had results that were actually useful.

### Word 2 Vec Training

#### German Idealism as a Test Case

We start by examining the texts of German Idealism to get a feel for what kind of parameters would work best.

In [21]:
def make_w2v(series, stopwords=[], size=200, window=5, min_count=5, workers=-1, 
             epochs=20, lowercase=True, sg=0, seed=17, cbow_mean=1, alpha=0.025,
             sample=0.001, use_bigrams=True, threshold=10, bigram_min=5):
  # turn the series into a list, lower it, clean it
    sentences = [sentence for sentence in series]
    if lowercase:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word.lower() for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)
    else:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)

  # incorporate bigrams
    if use_bigrams:
      bigram = Phrases(cleaned, min_count=bigram_min, threshold=threshold, delimiter=b' ')
      bigram_phraser = Phraser(bigram)
      tokens_list = []
      for sent in cleaned:
        tokens_ = bigram_phraser[sent]
        tokens_list.append(tokens_)
      cleaned = tokens_list
    else:
      cleaned = cleaned

  # build the model
    model = Word2Vec(cleaned, size=size, window=window, 
                     min_count=min_count, workers=workers, seed=seed, sg=sg,
                     cbow_mean=cbow_mean, alpha=alpha, sample=sample)
    model.train(series, total_examples=model.corpus_count, epochs=epochs)
    model_wv = model.wv
    
  # clear it to avoid unwanted transference
    del model

    return model_wv

In [22]:
gi_wv = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'], threshold=12)

We can check this model by trying out a few words. For that purpose we have a testing function that tries some common word combinations.

In [23]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [24]:
test_w2v_pos_neg(gi_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- joins (0.31005)
- tis (0.24875)
- immediacy (0.23487)
- feeling (0.22275)
- disappeared (0.22237)

Positive - ['self', 'consciousness']	Negative - []
- manifests itself (0.26899)
- deepest (0.23347)
- submitted (0.23027)
- positivity (0.22937)
- defend (0.22893)

Positive - ['dialectic']	Negative - []
- calls (0.26034)
- widespread (0.2585)
- steel spring (0.25831)
- ostensive (0.24845)
- deliberations (0.22695)

Positive - ['logic']	Negative - []
- frdm (0.25433)
- physician (0.25169)
- sunders (0.25144)
- therein (0.24754)
- ease (0.23451)



Although some of these make a modicum of sense a lot of them seem like just gibberish. Let's try messing with some parameters.



##### Trying Skip-gram instead of C-bow

In [25]:
# make a base model with the preset parameters
skip_gi_wv = make_w2v(series = df[df['school'] == 'german_idealism']['gensim_tokenized'], 
                         stopwords=[], sg=1, seed=0)

In [26]:
test_w2v(skip_gi_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- ive (0.30441)
- obscure (0.27291)
- manifestation (0.26622)
- import (0.244)
- ostensive (0.23673)

Positive - ['self', 'consciousness']	Negative - []
- drop (0.29426)
- definition (0.27958)
- ratiocination (0.2355)
- choice (0.23224)
- pleasure (0.22293)

Positive - ['dialectic']	Negative - []
- observe (0.2774)
- abroad (0.26245)
- paradoxical (0.251)
- lordship (0.24905)
- moral feeling (0.24531)

Positive - ['logic']	Negative - []
- factor (0.28102)
- renounced (0.26257)
- breazeale (0.23544)
- rerum (0.23437)
- nervous system (0.23188)



These seem mildy more sensible. Let's tweak the other parameters.

##### Parameter Testing

In [27]:
model_v1 = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'],
                       stopwords=[],
                       size=500,
                       window=5,
                       min_count=25,
                       epochs=10,
                       sg=1, 
                       seed=45)

len(model_v1.vocab)

2950

In [28]:
test_w2v(model_v1, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- rationality (0.15593)
- avoid (0.15046)
- association (0.14209)
- accord (0.13843)
- untrue (0.13657)

Positive - ['self', 'consciousness']	Negative - []
- expand (0.14267)
- merely (0.13895)
- syllogisms (0.13827)
- entity (0.13225)
- positing (0.13193)

Positive - ['dialectic']	Negative - []
- excludes (0.15812)
- completed (0.14345)
- important (0.1395)
- too (0.13569)
- primarily (0.13489)

Positive - ['logic']	Negative - []
- sure (0.16182)
- feeling (0.13781)
- preceding (0.13777)
- intuitions (0.13683)
- inherently (0.13238)



Despite tweaking parameters far and wide, it's difficult to get any results that are compellingly sensible. In most cases there are one or two terms in the similarity list that make some sense but others that are just strange or unconnected

#### Trying Another School

In [29]:
cm_w2v = make_w2v(df[df['school'] == 'communism']['gensim_tokenized'],
                       stopwords=[],
                       size=700,
                       window=10,
                       min_count=10,
                       epochs=25,
                       sg=1, 
                       seed=10)

type(cm_w2v)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [30]:
pairs_to_try=[(['material', 'conditions'], []),
              (['worker'], ['owner']),
              (['alienation', 'labor'], []),
              (['capital'], [])]

In [31]:
test_w2v(cm_w2v, pairs_to_try)

Positive - ['material', 'conditions']	Negative - []
- constant (0.14083)
- november (0.1261)
- revolutions (0.12528)
- italics (0.12458)
- just (0.12193)

Positive - ['worker']	Negative - ['owner']
- themselves (0.14149)
- slow (0.1371)
- rises (0.12916)
- every day (0.12628)
- streets (0.12155)

Positive - ['alienation', 'labor']	Negative - []
- motive power (0.14868)
- higher (0.13019)
- keeps (0.12603)
- certain extent (0.11582)
- wood (0.11516)

Positive - ['capital']	Negative - []
- set (0.13807)
- des (0.13375)
- be replaced (0.11977)
- specifically (0.11826)
- examined (0.1142)



Here the results were similar - a few words that made some sense and plenty that were just odd.  

### Transfer Learning with GloVe

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [32]:
# load the vectors. other vector sizes were used but yielded generally less sensible models
glove_file = datapath('/gdrive/MyDrive/Colab_Projects/Phil_NLP/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [46]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [34]:
# check out how GloVe works on our test pairs
test_w2v_pos_neg(glove_vectors, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- morality (0.82654)
- legal (0.82652)
- laws (0.81529)
- constitutional (0.80616)
- fundamental (0.80217)

Positive - ['self', 'consciousness']	Negative - []
- sense (0.83446)
- mind (0.79755)
- vision (0.78202)
- belief (0.78031)
- life (0.77984)

Positive - ['dialectic']	Negative - []
- hegelian (0.88376)
- dialectical (0.83417)
- dialectics (0.80672)
- materialist (0.77674)
- metaphysics (0.77488)

Positive - ['logic']	Negative - []
- reasoning (0.81405)
- intuitionistic (0.76531)
- concepts (0.75831)
- logical (0.75604)
- theory (0.75026)



Ok these make a lot more sense right from the start. But we want them to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

In [38]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [39]:
test_w2v(base_model_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- freedom (0.82611)
- happiness (0.81553)
- rule (0.80918)
- moral law (0.80705)
- morality (0.80598)

Positive - ['self', 'consciousness']	Negative - []
- self consciousness (0.90358)
- objectivity (0.86174)
- immediacy (0.85362)
- essence (0.85233)
- negativity (0.85204)

Positive - ['dialectic']	Negative - []
- antinomy (0.94207)
- method (0.90006)
- remainder (0.89822)
- morals (0.8973)
- critique (0.89044)

Positive - ['logic']	Negative - []
- pure reason (0.87167)
- metaphysics (0.86658)
- method (0.86523)
- philosophy (0.83807)
- science (0.81908)



We can immediately see that these make a lot more sense (and the similarity scores are a lot higher). 'Self' + 'consciousness' is rightly associated with 'self consciousness' and 'moral' + 'law' with 'moral law'. It even identifies the German Idealist tendency to unify logic and metaphysics. 

This is a massive improvement - these vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

In [40]:
def train_glove(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['school'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [ ]:
ph_model = train_glove(school='phenomenology', glove_vectors=glove_vectors)

In [47]:
test_w2v(ph_model, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- distribution (0.99305)
- alleged (0.99293)
- algebraic (0.99275)
- range (0.99253)
- interconnection (0.99223)

Positive - ['self', 'consciousness']	Negative - []
- nature (0.95029)
- potentiality (0.94732)
- certainty (0.93127)
- authentic (0.92803)
- existence (0.92441)

Positive - ['dialectic']	Negative - []
- inversion (0.98864)
- emergence (0.98773)
- mouth (0.98767)
- aid (0.98666)
- succession (0.98643)

Positive - ['logic']	Negative - []
- mathematics (0.98822)
- gathering (0.98441)
- primary (0.98374)
- apprehension (0.98373)
- disclosure (0.98349)



Using the phenomenology vectors on some central terms of German idealism once again yields some pretty compelling results, except for where the words are rarely used by the phenomenologists. This is to be expected. Let's try the word vectors on some central terms of phenomenology.

In [48]:
pairs_to_try = [(['perception'], []),
                (['dasein'], []),
                (['consciousness'], []),
                (['method'], []),]

test_w2v(ph_model, pairs_to_try)

Positive - ['perception']	Negative - []
- representation (0.9493)
- movement (0.94042)
- condition (0.93799)
- act (0.93541)
- attitude (0.922)

Positive - ['dasein']	Negative - []
- being (0.89689)
- itself (0.8852)
- truth (0.87685)
- consciousness (0.84428)
- understanding (0.8317)

Positive - ['consciousness']	Negative - []
- care (0.91553)
- truth (0.91257)
- future (0.90781)
- nature (0.90509)
- movement (0.90323)

Positive - ['method']	Negative - []
- necessity (0.96889)
- ontology (0.9638)
- spirit (0.96361)
- historicity (0.95475)
- metaphysics (0.95457)



These look pretty strong. Overall, the GloVe-traiend vectors seem to be an effective tool for revealing how a word is used by a school. 

#### Training on every school & author

To further explore this, we'll train w2v models in this way for each school and examine how each of them looks at the same word - 'philosophy.' We can use these in our future dashboard work.

In [49]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove(school, glove_vectors=glove_vectors)
  print(f'{school} completed')

plato completed
aristotle completed
empiricism completed
rationalism completed
analytic completed
continental completed
phenomenology completed
german_idealism completed
communism completed
capitalism completed


In [ ]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

	PLATO
----------------------
Positive - ['philosophy']	Negative - []
- relief (0.94646)
- springs (0.94496)
- friendship (0.94237)
- greece (0.93422)
- regime (0.93311)

	ARISTOTLE
----------------------
Positive - ['philosophy']	Negative - []
- respiration (0.91699)
- oratory (0.87146)
- shrillness (0.87085)
- holders (0.86975)
- memory (0.85944)

	EMPIRICISM
----------------------
Positive - ['philosophy']	Negative - []
- religion (0.93495)
- mankind (0.92261)
- doctrine (0.92259)
- inquiry (0.9155)
- faith (0.90897)

	RATIONALISM
----------------------
Positive - ['philosophy']	Negative - []
- person (0.95782)
- return (0.92869)
- death (0.92862)
- fall (0.92596)
- public (0.9258)

	ANALYTIC
----------------------
Positive - ['philosophy']	Negative - []
- philosophical (0.90815)
- hahn (0.85215)
- carnap (0.84692)
- semantics (0.84195)
- davidson (0.83312)

	CONTINENTAL
----------------------
Positive - ['philosophy']	Negative - []
- metaphysics (0.9689)
- unreason (0.95789)
- hist

Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. Continental thinkers mentioning unreason, analytic philosophers focusing on semantics, and phenomenologists associating philosophy with a method all track well. The ones that don't make sense are those that don't problematize the nature of philosophy to any great degree - capitalist thinkers aren't out there trying to discuss the nature of philosophy.

We'd also like vectors trained for each individual author. We can use these in our dashboard to enable intra-school comparisons of authors and generally allow for more fine-grained data exploration.

In [53]:
#@title Glove Training Function Modified for Authors
def train_glove_author(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['author'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [54]:
for author in df['author'].unique():
  w2v_dict[author] = train_glove_author(author, glove_vectors=glove_vectors)
  print(f'{author} completed')

Plato completed
Aristotle completed
Locke completed
Hume completed
Berkeley completed
Spinoza completed
Leibniz completed
Descartes completed
Malebranche completed
Russell completed
Moore completed
Wittgenstein completed
Lewis completed
Quine completed
Popper completed
Kripke completed
Foucault completed
Derrida completed
Deleuze completed
Merleau-Ponty completed
Husserl completed
Heidegger completed
Kant completed
Fichte completed
Hegel completed
Marx completed
Lenin completed
Smith completed
Ricardo completed
Keynes completed


With this finished - our next step is to train one on the entire corpus for use in classification.

#### Building a Model for the full Corpus

In [50]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv

As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [51]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])


	PHILOSOPHY CORPUS
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.8004)
- metaphysics (0.77228)
- religion (0.73747)
- science (0.72425)
- philosophical (0.71879)

	BASE GLOVE
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.88151)
- philosophical (0.84362)
- mathematics (0.83389)
- psychology (0.82387)
- sociology (0.81085)



This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

#### Finalized exporting

All in all, things look good, so let's export the vectors so that they can be used in our neural networks and in our dash app. 

In [59]:
all_text_wv.save_word2vec_format('/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/w2v_for_nn.bin')
all_text_wv.save('/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/w2v_for_nn.wordvectors')

In [60]:
for source in w2v_dict.keys():
  w2v_dict[source].save(f'/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_models/{source}_w2v.wordvectors')

And that's it! See our other notebooks for more of the modeling work. 